In [1]:
import xarray as xr
import geopandas as gpd
import rasterio as rio
import rioxarray as rxr
import pandas as pd
from pathlib import Path
import glob
import json

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
nc_fn = "/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/NCEP_reanalysis/skt.sfc.gauss.2022.nc"
ncep_reanalysis_dir = Path("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/NCEP_reanalysis")
data_dir = Path("/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/timeseries")
metadata = json.load(Path(data_dir, "processed", "metadata.json").open("r"))

target_param = "avg water temperature (C)"

In [3]:
def extract_skt(nc_filepath, lat, lon):
    ds = xr.open_dataset(nc_filepath)
    ds.coords['lon'] = (ds.coords['lon'] + 180) % 360 - 180
    ds = ds.sortby(ds.lon) 
    skt = ds['skt'].sel(lat=lat, lon=lon, method="nearest")
    skt = skt.to_dataframe()

    return skt

In [5]:
# extract data from the metadata to get reanalysis data
for station, value in metadata["stations"].items():
    if target_param in value["parameters"]:
        lat = value["latitude"]
        lon = value["longitude"] # adjust for values in the reanalysis data
        
        temperature_dfs = []
        for nc_filepath in glob.glob(str(ncep_reanalysis_dir / "*.nc")):
            temperature_dfs.append(extract_skt(nc_filepath, lat, lon))
        
        temperature_df = pd.concat(temperature_dfs, axis=0)
        # convert skt to celsius
        temperature_df["skt"] = temperature_df["skt"] - 273.15
        rename_dict = {"skt": "avg water temperature (C)"}
        temperature_df = temperature_df.rename(rename_dict, axis=1)
        
        temperature_df = temperature_df.groupby(level="time").mean().resample("D").mean()

        temperature_df.to_csv(data_dir / "NCEP_reanalysis" / f"{station}.csv")

        

In [2]:
# # download the data
# import urllib.request

# for year in range(1999, 2024):
#     urllib.request.urlretrieve(
#         f"https://downloads.psl.noaa.gov/Datasets/ncep.reanalysis/Dailies/surface_gauss/skt.sfc.gauss.{year}.nc",
#         f"/Users/gdarkwah/Library/CloudStorage/OneDrive-UW/01-Research/01-Hydrothermal History/Data/NCEP_reanalysis/skt.sfc.gauss.{year}.nc",
#     )